# Applying data minimization to a trained regression ML model

In this tutorial we will show how to perform data minimization for regression ML models using the minimization module.

We will show you applying data minimization to a different trained regression models.

## Load data
QI parameter determines which features will be minimized.

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import calcmetric as cm
import anonymize_module as am
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

dataset = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.5, random_state=14)

features = ['age', 'sex', 'bmi', 'bp',
                's1', 's2', 's3', 's4', 's5', 's6']
QI = ['age', 'bmi', 's2', 's5', 's6']

## Train DecisionTreeRegressor model

In [2]:
from apt.minimization import GeneralizeToRepresentative
from sklearn.tree import DecisionTreeRegressor

model1 = DecisionTreeRegressor(random_state=10, min_samples_split=2)
model1.fit(X_train, y_train)
print('Base model accuracy (R2 score): ', model1.score(X_test, y_test))

Base model accuracy (R2 score):  0.15014421352446072


## Run minimization
We will try to run minimization with only a subset of the features.

In [3]:
# note that is_regression param is True

minimizer1 = GeneralizeToRepresentative(model1, target_accuracy=0.7, is_regression=True,
                                    features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train1, x_test1, y_generalizer_train1, y_test1 = train_test_split(X_test, y_test,
                                                                test_size = 0.4, random_state = 38)

x_train_predictions1 = model1.predict(X_generalizer_train1)
minimizer1.fit(X_generalizer_train1, x_train_predictions1, features_names=features)
transformed1 = minimizer1.transform(x_test1, features_names=features)
print('Accuracy on minimized data: ', model1.score(transformed1, y_test1))
print('generalizations: ',minimizer1.generalizations)#%% md

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.108922
Improving accuracy
feature to remove: s5
Removed feature: s5, new relative accuracy: 0.505498
feature to remove: bmi
Removed feature: bmi, new relative accuracy: 0.716972
Accuracy on minimized data:  0.1116122925781402
generalizations:  {'ranges': {'age': [-0.07090024650096893, -0.043656209483742714, -0.041839939542114735, -0.03639113181270659, -0.01459590089507401, -0.012779632292222232, -0.009147093165665865, -0.0036982858437113464, 0.03989217430353165, 0.039892176166176796, 0.05623859912157059, 0.06713621318340302], 's2': [-0.0550188384950161, -0.0285577941685915, -0.024643437936902046, -0.02135537937283516, -0.013683241792023182, -0.006480826530605555, 0.009176596067845821, 0.023111702874302864, 0.02420772146433592, 0.02655633445829153, 0.039082273840904236], 's6': [-0.052854035049676895, -0.03835666086524725, -0.02593033987795

## Train linear regression model

In [4]:
from sklearn.linear_model import LinearRegression
from apt.minimization import GeneralizeToRepresentative

model2 = LinearRegression()
model2.fit(X_train, y_train)
print('Base model accuracy (R2 score): ', model2.score(X_test, y_test))

Base model accuracy (R2 score):  0.5080563960651392


## Run minimization
We will try to run minimization with only a subset of the features.

In [5]:
# note that is_regression param is True

minimizer2 = GeneralizeToRepresentative(model2, target_accuracy=0.7, is_regression=True,
                                    features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train2, x_test2, y_generalizer_train2, y_test2 = train_test_split(X_test, y_test,
                                                                test_size = 0.4, random_state = 38)

x_train_predictions2 = model2.predict(X_generalizer_train2)
minimizer2.fit(X_generalizer_train2, x_train_predictions2, features_names=features)
transformed2 = minimizer2.transform(x_test2, features_names=features)
print('Accuracy on minimized data: ', model2.score(transformed2, y_test2))
print('generalizations: ',minimizer2.generalizations)

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.201734
Improving accuracy
feature to remove: s5
Removed feature: s5, new relative accuracy: 0.292914
feature to remove: age
Removed feature: age, new relative accuracy: 0.291507
feature to remove: s2
Removed feature: s2, new relative accuracy: 0.947873
Accuracy on minimized data:  0.46523158691549726
generalizations:  {'ranges': {'bmi': [-0.0660245232284069, -0.06171327643096447, -0.048779530450701714, -0.04770171828567982, -0.036923596635460854, -0.022912041284143925, -0.01644516922533512, -0.015906263142824173, -0.009978296235203743, 0.007266696775332093, 0.022356065921485424, 0.028822937980294228, 0.04499012045562267, 0.053073709830641747, 0.10103634744882584], 's6': [-0.07356456853449345, -0.052854035049676895, -0.048711927607655525, -0.046640874817967415, -0.044569820165634155, -0.0383566590026021, -0.021788232028484344, -0.017646125

## Calculate anonymization metrics
We'll measure different metrics for the transformed dataset


In [6]:
df = pd.DataFrame(transformed2, columns=features)
df['sensval'] = y_test2

# find maximum k-value for which k-anonymity is still satisfied
cm.find_k_anonymity(df, QI)

Dataset satisfies 1-anonymity
Dataset does not satisfy 2-anonymity
Dataset satisfies maximum 1-anonymity


1

## K-Anonymize
As seen, the dataset is not anonymized
To anonymize the dataset, call k-anonymize function

In [7]:
k_anonymdf = am.anonymize_k_anonymity(df, QI, 'sensval', k=10)
cm.find_k_anonymity(k_anonymdf, QI)


=========Start anonymization process:
Partition the dataset:
8 partitions created.
Changing quasi-identifiers values with the aggregation of their partition
Showcasing some of the anonymized partition:
Partition 7
+----+-----------+------------+-----------+------------+------------+-----------+-------------+-------------+-----------+-----------+-----------+
|    |       age |        sex |       bmi |         bp |         s1 |        s2 |          s3 |          s4 |        s5 |        s6 |   sensval |
|----+-----------+------------+-----------+------------+------------+-----------+-------------+-------------+-----------+-----------+-----------|
|  0 | 0.0380759 |  0.0506801 | 0.0161586 | -0.0641989 | 0.0699806  | 0.0560967 | -0.0397192  |  0.07121    | 0.0401722 | 0.0210135 |       116 |
|  4 | 0.0380759 | -0.0446416 | 0.0161586 | -0.0538703 | 0.0631008  | 0.0560967 | -0.0102661  |  0.0343089  | 0.0401722 | 0.0210135 |       164 |
| 11 | 0.0380759 | -0.0446416 | 0.0161586 |  0.0631866 |

10